In [179]:
import scanpy as sc
import scvelo as scv
import numpy as np
import os
from anndata import AnnData as ad

In [180]:
sc.settings.datasetdir="../../datasets/raw/"

Read dataset 

In [181]:
adata_raw = sc.datasets.pbmc3k()
adata_raw

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [182]:
adata_processed = sc.datasets.pbmc3k_processed()

In [183]:
adata_processed.X = adata_raw[list(adata_processed.obs.index), list(adata_processed.var.index)].X
adata_processed.layers["X_counts"] = adata_processed.X.copy()
adata_processed.obs = adata_processed.obs.rename(columns={"louvain": "cell_type"})

In [184]:
sc.pp.log1p(adata_processed)

In [185]:
sc.tl.pca(adata_processed)
sc.pp.neighbors(adata_processed)
sc.tl.umap(adata_processed)

Save

In [173]:
ad.write_h5ad(adata_processed, "../../datasets/processed/pbmc3k.h5ad")

In [187]:
adata = sc.read_h5ad("../../datasets/processed/pbmc3k.h5ad")